## **FUNCTIONS, VARS, INIT**

In [1]:
home_path=!pwd
thisnb=home_path[0]+"/szechuan_volatility.ipynb"

## **IMPORTS & INIT**

In [2]:
# Data Science Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_bokeh
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show

# DFIR Imports
from timesketch_api_client import client
from timesketch_api_client import config
import timesketch_api_client

# DS4N6
from ds4n6_lib.common import analysis
from ds4n6_lib.common import anl
from ds4n6_lib.common import whatis
from ds4n6_lib.unx    import xgrep
from ds4n6_lib.gui    import read_data_gui
from ds4n6_lib.gui    import xmenu
from ds4n6_lib.gui    import xdisplay

import ds4n6_lib.d4          as d4
import ds4n6_lib.common      as d4com
import ds4n6_lib.autoruns    as d4atrs
import ds4n6_lib.fstl        as d4fstl
import ds4n6_lib.plaso       as d4pl
import ds4n6_lib.kansa       as d4ksa
import ds4n6_lib.kape        as d4kp
import ds4n6_lib.evtx        as d4evtx
import ds4n6_lib.unx         as d4unx
import ds4n6_lib.utils       as d4utl
import ds4n6_lib.volatility  as d4vol
import ds4n6_lib.gui         as d4gui

# Standard python imports
from time import sleep
import datetime
import time
import re
import os.path
import nbformat as nbf
import calendar
import glob
import xmltodict
import untangle
import importlib
import logging

# Data Save/Restore
import dill
import pickle
import gdown

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=UserWarning)

## **READ DATA**

In [3]:
url = 'https://drive.google.com/uc?id=1nRxNTpPuZTzvydCwjfxIhtGa5OdaLDFw'
of = 'volatility.tgz'
gdown.download(url, of, quiet=False)
gdown.extractall("volatility.tgz", to="volatility/")

Downloading...
From: https://drive.google.com/uc?id=1nRxNTpPuZTzvydCwjfxIhtGa5OdaLDFw
To: /home/jovyan/volatility.tgz
15.2MB [00:00, 52.7MB/s]


['volatility/vol-wndscan.out.txt.psv',
 'volatility/vol-wintree.out.txt.psv',
 'volatility/vol-windows.out.txt.psv',
 'volatility/vol-vmwareinfo.out.txt.psv',
 'volatility/vol-vboxinfo.out.txt.psv',
 'volatility/vol-vadwalk.out.txt.psv',
 'volatility/vol-vadtree.out.txt.psv',
 'volatility/vol-vadinfo.out.txt.psv',
 'volatility/vol-usnparser.out.txt.psv',
 'volatility/vol-userhandles.out.txt.psv',
 'volatility/vol-userassist.out.txt.psv',
 'volatility/vol-unloadedmodules.out.txt.psv',
 'volatility/vol-truecryptsummary.out.txt.psv',
 'volatility/vol-truecryptpassphr.out.txt.psv',
 'volatility/vol-truecryptmaster.out.txt.psv',
 'volatility/vol-timers.out.txt.psv',
 'volatility/vol-timeliner.out.txt.psv',
 'volatility/vol-threads.out.txt.psv',
 'volatility/vol-thrdscan.out.txt.psv',
 'volatility/vol-symlinkscan.out.txt.psv',
 'volatility/vol-svcscan.out.txt.psv',
 'volatility/vol-ssdt.out.txt.psv',
 'volatility/vol-sockscan.out.txt.psv',
 'volatility/vol-sockets.out.txt.psv',
 'volatility/

In [4]:
%%time

d4.debug = 0

# Read raw data
read_data_gui(tool="volatility", rootpath=home_path[0]+"/volatility", prefix="vol-", notebook_file=thisnb, ext=".out.txt.psv", separator="|", drop_non_ham_cols=False, use_pickle=False)

- Searching notebook for saved input file / folder (_volatilityd)
  + Found: /home/jovyan/volatility

- Reading file(s):
  + INFO: 90 files found to read.

  + [1/90] vol-getsids.out.txt.psv                                      -> getsids                                 [1117]
  + [2/90] vol-ldrmodules.out.txt.psv                                   -> ldrmodules                              [171]
  + [3/90] vol-dlllist.out.txt.psv                                      -> dlllist                                 [1026]
  + [4/90] vol-iehistory.out.txt.psv                                    -> iehistory                               [Empty File]
  + [5/90] vol-threads.out.txt.psv                                      -> threads                                 [11539]
  + [6/90] vol-privs.out.txt.psv                                        -> privs                                   [2754]
  + [7/90] vol-getservicesids.out.txt.psv                               -> getservicesids                 

/srv/conda/envs/notebook/lib/python3.7/site-packages/ds4n6_lib/gui.py:127: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  read_selected_file(dummy, compname=compname, f2read=f2read, nbsave_prefix=nbsave_prefix, **kwargs)


[80366]
  + [33/90] vol-vadwalk.out.txt.psv                                      -> vadwalk                                 [ERROR]
  + [34/90] vol-clipboard.out.txt.psv                                    -> clipboard                               [0]
  + [35/90] vol-multiscan.out.txt.psv                                    -> multiscan                               [Empty File]
  + [36/90] vol-timeliner.out.txt.psv                                    -> timeliner                               [1965]
  + [37/90] vol-driverscan.out.txt.psv                                   -> driverscan                              [83]
  + [38/90] vol-consoles.out.txt.psv                                     -> consoles                                [0]
  + [39/90] vol-psxview.out.txt.psv                                      -> psxview                                 [98]
  + [40/90] vol-patcher.out.txt.psv                                      -> patcher                                 [0]
  + [41/90] vo

In [5]:
voldfs = d4.out

In [6]:
# Automatically created - DO NOT EDIT OR REMOVE unless you want to change the file to read (in that case, remove this cell)
# 2021-04-07 16:04:56.499470
# _volatilityd_f2read = "/home/jovyan/volatility"

## **ANALYSIS**

In [7]:
xdisplay()

#### Pandas options

IntText(value=60, description='display.max_rows: ', style=DescriptionStyle(description_width='initial'))

IntText(value=10, description='display.min_rows: ', style=DescriptionStyle(description_width='initial'))

IntText(value=20, description='display.max_columns: ', style=DescriptionStyle(description_width='initial'))

Dropdown(description='display.colheader_justify: ', options=('right', 'left'), style=DescriptionStyle(descript…

Checkbox(value=True, description='display.expand_frame_repr', indent=False, style=DescriptionStyle(description…

In [8]:
xmenu(voldfs)

**DataFrame visualization menu:**

Box(children=(Dropdown(description='Select DataFrame', layout=Layout(width='550px'), options=('Select DataFram…

Output(layout=Layout(overflow='auto', width='99%'))

Output(layout=Layout(overflow='auto', width='99%'))

In [9]:
d4gui.xanalysis(voldfs)

**Analisys explorer:**

Dropdown(description='Available analysis types: ', layout=Layout(width='550px'), options=(('Select analysis ty…

Output(layout=Layout(overflow='auto', width='99%'))